## Fra Jacob

· Jeg arbejder med dette datasæt: ”bodies_df_exposure.csv”. Er det det nyeste?

· Da vi sås sidst, var du vistnok i gang med at konstruere et datasæt, der aggregerer forskellige udefrakommende variable (fra PRIO vistnok, fx ”deaths_a”, ”deaths_b”, ”deaths_civilians”, osv) inden for tidsperioden, måned? Er det korrekt forstået? Hvad er status på det? Ville det måske give mening at aggregere op til noget mere fintmasket temporalt? Vi har jo data på dagen, hvor billedet er taget?

· Hvis ja, er dette datasæt så allerede aggregeret til samme grid-cell størrelse som i bodies_df_exposure? Eller kan man i den proces konstruere et mere fintmasket grid?

· Kan man også merge nogle variable ind fra UCDP, fx dem jeg nævnte nedenfor (bdist3 capdist gcp_mer gcp_ppp  excluded nlights_mean, og så videre). Hvis ja, skal man så merge på gid og year? Og hvis ja, findes disse variable så i en mere fintmasket opløsning end data i bodies_df_exposure?

· I forhold til datasættet ”bodies_df_exposure”: Det ser ud som om, der er flere unikke byer (37) end grid cells (21). Kan det være rigtigt? Og, mere konstruktivt, kan vi måske bruge dét til at få en mere fintmasket geografisk opløsning? Eventuelt bare bruge byer i stedet for grid som tværsnits-enhed?


--- Svar: må være en csv fejl....

· Nogle af de relevante variable i ”bodies_df_exposure” indeholder lange strings – fx indeholder ”person_mean” 1399 observationer med værdien ”4 BCT” og 595 observationer med værdien ”and is currently planning to move it”. Jeg tror, at det sker fordi headline-variable flyder ind i andre variable. Kan du eventuelt tjekke? Én let løsning kunne jo være at lave samme datasæt igen uden headline. Det kan jeg ikke selv – efter import i stata er den allerede gal.

· Jeg har undret mig over variablen ”headline”. Der er 13.849 headlines i alt, men der er mange der går igen. Fx er der 1584 headlines, der hedder ” New Iraqi Army Training”. Jeg ville have troet, at headlines var rimeligt unikke, så måske de dækker over noget andet end en overskrift i en avis. Kan du afklare?

In [1]:
# use geo_env_2022
import os
import numpy as np
import pandas as pd # original 1.2.3
import pickle
import urllib.request

import matplotlib.pyplot as plt

In [2]:
def get_data():

    image_data_dir = '/home/simon/Documents/Bodies/data/OD_dataframes_compiled/'

    pkl_file = open(f'{image_data_dir}bodies_df_exposure.pkl', 'rb')

    bodies_df = pickle.load(pkl_file)
    pkl_file.close()

    prio_data_dir = '/home/simon/Documents/Bodies/data/PRIO/'

    pkl_file = open(f'{prio_data_dir}full_interpl_df_exposure.pkl', 'rb')

    prio_df = pickle.load(pkl_file)
    pkl_file.close()

    return(bodies_df, prio_df)



In [3]:
def merge_dfs(bodies_df, prio_df):

    # drop this as it is making trouble.
    bodies_df['year'] = bodies_df['new_year'] # correct year feature
    bodies_df.drop(columns=['headline', 'new_year'], inplace=True)
    bodies_df.dropna(subset=['gid', 'month_id', 'year'], inplace= True) # there  

    merge_on = ['gid', 'month_id', 'year']

    for i in merge_on:
        bodies_df.loc[:,i] = bodies_df.loc[:,i].astype('int')

    bodies_df.reset_index(drop=True, inplace= True)

    bodies_prio_df = bodies_df.merge(prio_df, on = merge_on, how='left', suffixes=('_left', '_right'))

    # just drop the lefts and rename the rights (not that impoortant which one)
    bodies_prio_df.drop(columns=['log_best_left', 'log_high_left', 'log_low_left', 'tce_left', 'tsce_left'], inplace=True)
    bodies_prio_df.rename(columns = {'log_best_right':'log_best', 'log_high_right':'log_high' , 'log_low_right': 'log_low', 'tce_right': 'tce', 'tsce_right' : 'tsce'}, inplace = True)

    return(bodies_prio_df)

In [4]:
bodies_df, prio_df = get_data()
bodies_prio_df = merge_dfs(bodies_df, prio_df)

data_dir = '/home/simon/Documents/Bodies/data/done_dfs/'

bodies_prio_df.to_pickle(f'{data_dir}bodies_df_2022_v1_0.pkl')
bodies_prio_df.to_csv(f'{data_dir}bodies_df_2022_v1_0.csv')

In [5]:
bodies_prio_df

,person_mean,person_median,person_fasterR50,person_fasterR101,person_fasterX101,person_retinaR50,person_retinaR101,child_mean,child_median,child_retinaR50,...,prec_gpcp_observed,urban_ih_observed,cmr_mean,imr_mean,mountains_mean,ttime_mean,tce,tce_std,tsce,tsce_std
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,...,False,False,15.90000,1020.000,0.0,58.67639,1.956879,0.66848,1.051318,0.756553
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,...,False,False,15.90000,1020.000,0.0,58.67639,2.914739,0.66848,1.355943,0.756553
2,1.6,2.0,2.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,...,False,False,15.90000,1020.000,0.0,58.67639,3.367234,0.66848,1.978720,0.756553
3,2.8,3.0,2.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,...,False,False,15.87917,1014.583,0.0,415.26140,3.018411,0.66848,1.516200,0.756553
4,1.8,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,...,False,False,15.87917,1014.583,0.0,415.26140,3.018411,0.66848,1.516200,0.756553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152183,0.6,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,True,False,15.90000,1020.000,0.0,168.32170,-0.022490,0.66848,-0.016555,0.756554
152184,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,...,True,False,15.90000,1020.000,0.0,168.32170,-0.022490,0.66848,-0.016555,0.756554
152185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,True,False,15.90000,1020.000,0.0,168.32170,-0.022490,0.66848,-0.016555,0.756554
152186,4.8,5.0,5.0,5.0,5.0,5.0,4.0,1.5,1.5,2.0,...,True,False,15.90000,1020.000,0.0,168.32170,-0.022490,0.66848,-0.016555,0.756554
